In [6]:
from tqdm import tqdm
from collections import Counter
from rdkit import Chem
from MACCSkeys import smartsPatts

In [9]:
rxncounts_per_key = Counter()
maccs_keys_to_skip = set([1, 44, 125, 166])

num_rxns = sum(1 for line in open("../data/raw/train.txt", "r"))

with open("../data/raw/train.txt", "r") as train_dataset:
    for reaction in tqdm(train_dataset, total = num_rxns):

        lhs, rhs = reaction.split(">>")
        lhs_mol = Chem.MolFromSmiles(lhs)
        rhs_mol = Chem.MolFromSmiles(rhs)

        for maccs_key in smartsPatts:

            if maccs_key in maccs_keys_to_skip:
                continue

            maccs_smarts = smartsPatts[maccs_key][0]
            maccs_pattern = Chem.MolFromSmarts(maccs_smarts)

            if not maccs_pattern:
                print(maccs_smarts, maccs_key)

            if (lhs_mol.HasSubstructMatch(maccs_pattern)
                or rhs_mol.HasSubstructMatch(maccs_pattern)):
                rxncounts_per_key.update({maccs_key: 1})


100%|██████████| 409035/409035 [34:24<00:00, 198.16it/s]


In [11]:
import pickle

# Store data (serialize)
with open('rxncounts_per_key.pickle', 'wb') as handle:
    pickle.dump(rxncounts_per_key, handle, protocol=pickle.HIGHEST_PROTOCOL)